In [1]:
import torch

import torch.nn as nn
from keras.regularizers import l1,l2
from keras.models import Sequential,Model
from keras.layers import Embedding,Input,Dense,Flatten,Lambda, Multiply, Concatenate


2022-12-15 16:52:07.158290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 16:52:07.258079: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
torch.load('/home/vuhoang181/Documents/code_base/APR-PyTorch/output/bpr.pt')

OrderedDict([('W',
              tensor([[ 0.1027, -0.2571,  0.2280,  ..., -0.3319,  0.1107, -0.0547],
                      [-0.2378,  0.1023, -0.2476,  ..., -0.1183,  0.2373,  0.3497],
                      [ 0.1866, -0.2054, -0.2772,  ..., -0.2186,  0.2774,  0.2513],
                      ...,
                      [ 0.1794, -0.2494,  0.2269,  ..., -0.2170,  0.0641,  0.0161],
                      [ 0.2467, -0.1777,  0.1404,  ..., -0.2994, -0.1175, -0.0339],
                      [-0.1238,  0.1703, -0.1914,  ...,  0.0497,  0.2560, -0.1212]])),
             ('H',
              tensor([[-0.1210, -0.0994, -0.2235,  ..., -0.2647, -0.2383, -0.1061],
                      [-0.1862,  0.1676, -0.3127,  ..., -0.2307, -0.2465, -0.3151],
                      [-0.2041,  0.2428, -0.3161,  ..., -0.2608, -0.1408, -0.2574],
                      ...,
                      [-0.2339, -0.0100,  0.1009,  ..., -0.0604, -0.1932, -0.1700],
                      [ 0.0182,  0.1940, -0.2179,  ..., -0.1539, 

In [ ]:
def get_model(num_users,num_items,mf_dim=10,layers=[10],reg_layers=[0,0,0,0],reg_mf=0):
    num_layer=len(layers)

    user_input=Input(shape=(1,),dtype='int32')
    item_input_pos=Input(shape=(1,),dtype='int32')
    item_input_neg = Input(shape=(1,), dtype='int32')

    ## in this context each point is projected to have 10 dimensional coordinates?
    MF_embedding_user=Embedding(input_dim=num_users,output_dim=mf_dim,embeddings_initializer='random_normal',
                                name='mf_user_embedding',embeddings_regularizer=l2(reg_mf),input_length=1)
    MF_embedding_item = Embedding(input_dim=num_items, output_dim=mf_dim, embeddings_initializer='random_normal',
                                  name='mf_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)
    MLP_embedding_user=Embedding(input_dim=num_users,output_dim=layers[0],embeddings_initializer='random_normal',
                                 name='mlp_user_embedding', embeddings_regularizer=l2(reg_mf),input_length=1)
    MLP_embedding_item = Embedding(input_dim=num_items, output_dim=layers[0], embeddings_initializer='random_normal',
                                   name='mlp_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)

    mf_user_latent=Flatten()(MF_embedding_user(user_input))
    mf_item_latent_pos=Flatten()(MF_embedding_item(item_input_pos))
    mf_item_latent_neg = Flatten()(MF_embedding_item(item_input_neg))

    ## merge = deprecated use keras.layers.Concatenate(axis=-1) instead
    prefer_pos = merge([mf_user_latent, mf_item_latent_pos], mode='mul')
    prefer_neg = merge([mf_user_latent, mf_item_latent_neg], mode='mul')
    ## convert negative layer to negative, lambda layers should be re-written as subclass layer if too complex (eg: multiply by scale?)
    prefer_neg = Lambda(lambda x: -x)(prefer_neg)
    ## basically just merge 2 layer
    mf_vector = merge([prefer_pos, prefer_neg], mode='concat')

    ## flat matrix to an array
    mlp_user_latent=Flatten()(MLP_embedding_user(user_input))
    mlp_item_latent_pos=Flatten()(MLP_embedding_item(item_input_pos))
    mlp_item_latent_neg=Flatten()(MLP_embedding_item(item_input_neg))
    mlp_item_latent_neg=Lambda(lambda x:-x)(mlp_item_latent_neg)
    mlp_vector=merge([mlp_user_latent,mlp_item_latent_pos,mlp_item_latent_neg],mode='concat')
    for idx in range(1,num_layer):
        #set up hidden layer of the network, why tanh and have to regularize?? L1 consider weight, l2 consider square of weight
        layer=Dense(layers[idx],kernel_regularizer=l2(0.0000),activation='tanh',name="layer%d" %idx)
        mlp_vector=layer(mlp_vector)

    ## concatenation of NBPR layer and DNCR layer
    predict_vector=merge([mf_vector,mlp_vector],mode='concat')


    #set up prediction --> final layer, sigmoid activate to give binary output
    prediction=Dense(1,activation='sigmoid',kernel_initializer='lecun_uniform',name='prediction')(predict_vector)
    model=Model(inputs=[user_input,item_input_pos,item_input_neg],outputs=prediction)

    return model

In [5]:
user_input=Input(shape=(1,),dtype='int32')
item_input_pos=Input(shape=(1,),dtype='int32')
item_input_neg = Input(shape=(1,), dtype='int32')

In [7]:
mf_dim=10
layers=[10]
num_layer=len(layers)
reg_layers=[0,0,0,0]
reg_mf=0

In [14]:
MF_embedding_user=Embedding(input_dim=2000,output_dim=mf_dim,embeddings_initializer='random_normal',
                                name='mf_user_embedding',embeddings_regularizer=l2(reg_mf),input_length=1)
MF_embedding_item = Embedding(input_dim=50000, output_dim=mf_dim, embeddings_initializer='random_normal',
                                  name='mf_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)
MLP_embedding_user=Embedding(input_dim=2000,output_dim=layers[0],embeddings_initializer='random_normal',
                                 name='mlp_user_embedding', embeddings_regularizer=l2(reg_mf),input_length=1)
MLP_embedding_item = Embedding(input_dim=50000, output_dim=layers[0], embeddings_initializer='random_normal',
                                   name='mlp_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)

In [15]:
mf_user_latent=Flatten()(MF_embedding_user(user_input))
mf_item_latent_pos=Flatten()(MF_embedding_item(item_input_pos))
mf_item_latent_neg = Flatten()(MF_embedding_item(item_input_neg))

2022-12-14 23:27:03.991416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
## merge = deprecated use keras.layers.Concatenate(axis=-1) instead
prefer_pos = Multiply()([mf_user_latent, mf_item_latent_pos])
prefer_neg = Multiply()([mf_user_latent, mf_item_latent_neg])
## convert negative layer to negative, lambda layers should be re-written as subclass layer if too complex (eg: multiply by scale?)
prefer_neg = Lambda(lambda x: -x)(prefer_neg)
## basically just merge 2 layer
mf_vector = Concatenate()([prefer_pos, prefer_neg])

In [27]:
mlp_user_latent=Flatten()(MLP_embedding_user(user_input))
mlp_item_latent_pos=Flatten()(MLP_embedding_item(item_input_pos))
mlp_item_latent_neg=Flatten()(MLP_embedding_item(item_input_neg))
mlp_item_latent_neg=Lambda(lambda x:-x)(mlp_item_latent_neg)
mlp_vector=Concatenate()([mlp_user_latent,mlp_item_latent_pos,mlp_item_latent_neg])
for idx in range(1,num_layer):
    #set up hidden layer of the network, why tanh and have to regularize?? L1 consider weight, l2 consider square of weight
    layer=Dense(layers[idx],kernel_regularizer=l2(0.0000),activation='tanh',name="layer%d" %idx)
    mlp_vector=layer(mlp_vector)

In [37]:
predict_vector=Concatenate()([mf_vector,mlp_vector])


#set up prediction --> final layer, sigmoid activate to give binary output
prediction=Dense(3,kernel_initializer='lecun_uniform',name='prediction')(predict_vector)
model=Model(inputs=[user_input,item_input_pos,item_input_neg],outputs=prediction)

In [38]:
prediction

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'prediction')>

In [40]:
layer=Dense(1,kernel_regularizer=l2(0.0000),activation='tanh')
mlp_vector=layer(mf_vector)

In [41]:
mlp_vector

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

In [36]:
layers

[10]

In [25]:
num_layer=len(layers)

In [26]:
num_layer

1

In [5]:
W = nn.Parameter(torch.empty(2000, 64))  # User embedding
H = nn.Parameter(torch.empty(10000, 64))  # Item embedding
nn.init.xavier_normal_(W.data)
nn.init.xavier_normal_(H.data)

tensor([[ 0.0071, -0.0003, -0.0036,  ..., -0.0109, -0.0021, -0.0093],
        [-0.0168,  0.0075,  0.0166,  ...,  0.0210,  0.0088,  0.0050],
        [ 0.0069,  0.0208, -0.0190,  ...,  0.0083,  0.0040, -0.0226],
        ...,
        [ 0.0188, -0.0042,  0.0058,  ..., -0.0114,  0.0023,  0.0108],
        [ 0.0068, -0.0273,  0.0137,  ..., -0.0128,  0.0244,  0.0166],
        [ 0.0157, -0.0429, -0.0156,  ..., -0.0312, -0.0007,  0.0045]])

In [34]:
u=W[1,:]
i=H[1,:]
j=H[2,:]

In [26]:
u.retain_grad()
i.retain_grad()
j.retain_grad()

In [22]:
torch.mul(u, i).sum()

tensor(-0.0072, grad_fn=<SumBackward0>)

In [19]:
u

tensor([ 0.0308,  0.0214, -0.0337, -0.0007,  0.0668,  0.0039,  0.0473, -0.0047,
        -0.0129, -0.0313, -0.0083,  0.0202,  0.0343, -0.0094, -0.0246,  0.0225,
        -0.0211,  0.0477, -0.0036, -0.0222, -0.0022, -0.0409, -0.0026, -0.0114,
         0.0423, -0.0686,  0.0195,  0.0083, -0.0347,  0.0004, -0.0054,  0.0049,
        -0.0268, -0.0576, -0.0233, -0.0235, -0.0006,  0.0016, -0.0176,  0.0523,
        -0.0086,  0.0087,  0.0148,  0.0087,  0.0461, -0.0197,  0.0121, -0.0110,
        -0.0299, -0.0086, -0.0063, -0.0063,  0.0055, -0.0220, -0.0304, -0.0149,
        -0.0629,  0.0318,  0.0386, -0.0172, -0.0283, -0.0047, -0.0392, -0.0386],
       grad_fn=<SliceBackward0>)

In [23]:
import torch.nn.functional as F


In [24]:
F.logsigmoid(torch.mul(u, i).sum()).sum()

tensor(-0.6968, grad_fn=<SumBackward0>)

In [29]:
u

tensor([ 0.0308,  0.0214, -0.0337, -0.0007,  0.0668,  0.0039,  0.0473, -0.0047,
        -0.0129, -0.0313, -0.0083,  0.0202,  0.0343, -0.0094, -0.0246,  0.0225,
        -0.0211,  0.0477, -0.0036, -0.0222, -0.0022, -0.0409, -0.0026, -0.0114,
         0.0423, -0.0686,  0.0195,  0.0083, -0.0347,  0.0004, -0.0054,  0.0049,
        -0.0268, -0.0576, -0.0233, -0.0235, -0.0006,  0.0016, -0.0176,  0.0523,
        -0.0086,  0.0087,  0.0148,  0.0087,  0.0461, -0.0197,  0.0121, -0.0110,
        -0.0299, -0.0086, -0.0063, -0.0063,  0.0055, -0.0220, -0.0304, -0.0149,
        -0.0629,  0.0318,  0.0386, -0.0172, -0.0283, -0.0047, -0.0392, -0.0386],
       grad_fn=<SliceBackward0>)

In [62]:
mlp_vector=torch.stack((u,i,j))

In [47]:
Dense(1,kernel_regularizer=l2(0.0000),activation='tanh',name='final')(mlp_vector)

AttributeError: 'torch.Size' object has no attribute 'rank'

In [77]:
m = torch.nn.Linear(mlp_vector.shape[1], 1)

In [51]:
import torch
import torch.nn as nn

x = torch.tensor([[1.0, -1.0],
                  [0.0,  1.0],
                  [0.0,  0.0]])

in_features = x.shape[1]  # = 2
out_features = 2

m = nn.Linear(in_features, out_features)

In [87]:
m(mlp_vector)

tensor([[-0.0586],
        [-0.0656],
        [-0.0755]], grad_fn=<AddmmBackward0>)

In [90]:
m1 = nn.Linear(64, 32)
m2 = nn.Linear(32,1)

In [91]:
m2(m1(mlp_vector))

tensor([[-0.0560],
        [-0.0568],
        [-0.0535]], grad_fn=<AddmmBackward0>)

In [92]:
linear_relu_stack = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

In [95]:
linear_relu_stack(mlp_vector)

tensor([[0.1656],
        [0.1633],
        [0.1632]], grad_fn=<AddmmBackward0>)

In [96]:
linear_relu_stack(mlp_vector)[1]

tensor([0.1633], grad_fn=<SelectBackward0>)

In [1]:
import math
import os
import random
import pickle
import argparse
from collections import deque
import time
from datetime import timedelta
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import IterableDataset, DataLoader, get_worker_info
import matplotlib.pyplot as plt


## time the process
def get_time_dif(start_time):
    """get the running time"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))


## set up the u,i,j triplet for BPR framework
class GetTriplePair(IterableDataset):
    def __init__(self, item_size, user_list, pair, shuffle, num_epochs):
        self.item_size = item_size
        self.user_list = user_list
        self.pair = pair
        self.shuffle = shuffle
        self.num_epochs = num_epochs

    def __iter__(self):
        self.example_size = self.num_epochs * len(self.pair)
        self.example_index_queue = deque([])
        self.seed = 0
        self.start_list_index = None
        self.num_workers = 1
        self.index = 0
        return self

    def __next__(self):
        if self.index >= self.example_size:
            raise StopIteration
        # If `example_index_queue` is used up, replenish this list.
        while len(self.example_index_queue) == 0:
            index_list = list(range(len(self.pair)))
            if self.shuffle:
                random.Random(self.seed).shuffle(index_list)
                self.seed += 1
            if self.start_list_index is not None:
                index_list = index_list[self.start_list_index::self.num_workers]

                # Calculate next start index
                self.start_list_index = (self.start_list_index + (self.num_workers - (len(self.pair) % self.num_workers))) % self.num_workers
            self.example_index_queue.extend(index_list)
        result = self._example(self.example_index_queue.popleft())
        self.index += self.num_workers
        return result

    def _example(self, idx):
        u = self.pair[idx][0]
        i = self.pair[idx][1]
        j = np.random.randint(self.item_size)
        while j in self.user_list[u]:
            j = np.random.randint(self.item_size)
        return u, i, j

## chunk to define matrix factorization part
class MF(nn.Module):
    def __init__(self, user_size, item_size, dim, reg, reg_adv, eps):
        super().__init__()
        ##init the embedding for U and I
        self.W = nn.Parameter(torch.empty(user_size, dim))  # User embedding
        self.H = nn.Parameter(torch.empty(item_size, dim))  # Item embedding
        nn.init.xavier_normal_(self.W.data)
        nn.init.xavier_normal_(self.H.data)
        self.reg = reg
        self.user_size = user_size
        self.item_size = item_size
        self.dim = dim
        self.reg_adv = reg_adv
        self.eps = eps
        self.update_u = None
        self.update_i = None
        self.update_j = None

## forward cal
    def forward(self, u, i, j, epoch):
        """Return loss value.

        Args:
            u(torch.LongTensor): tensor stored user indexes. [batch_size,]
            i(torch.LongTensor): tensor stored item indexes which is prefered by user. [batch_size,]
            j(torch.LongTensor): tensor stored item indexes which is not prefered by user. [batch_size,]
            epoch

        Returns:
            torch.FloatTensor
        """
        ##u,i,j respectively, each is a vector of dim embedding (default = 64)
        u = self.W[u, :]
        i = self.H[i, :]
        j = self.H[j, :]

        ## Enables this Tensor to have their grad populated during backward(), convert any non-leaf tensor into a leaf tensor,
        ##https://stackoverflow.com/questions/73698041/how-retain-grad-in-pytorch-works-i-found-its-position-changes-the-grad-result
        u.retain_grad()
        u_clone = u.data.clone()
        i.retain_grad()
        i_clone = i.data.clone()
        j.retain_grad()
        j_clone = j.data.clone()

        ## mf, dot product of user with pos/neg item
        x_ui = torch.mul(u, i).sum(dim=1)
        x_uj = torch.mul(u, j).sum(dim=1)


        #similar to clip value, find diff between ui and uj
        x_uij =torch.clamp(x_ui - x_uj,min=-80.0,max=1e8)
        #logsigmoid this is equivalent to equation 1 in the paper (classic loss of bpr)
        log_prob = F.logsigmoid(x_uij).sum()
        # regularization = lambda * l2 norm of u, i, j
        regularization = self.reg * (u.norm(dim=1).pow(2).sum() + i.norm(dim=1).pow(2).sum() + j.norm(dim=1).pow(2).sum())

        ## original bpr loss,
        loss = -log_prob + regularization


        ## add adv training after a certain number of epochs, here is the part which we add hypernet module
        if epoch not in range(args.epochs, args.adv_epoch + args.epochs):
            """Normal training"""
            loss.backward()
            return loss

        else:
            """Adversarial training:
                    1.Backward to get grads
                    2.Construct adversarial perturbation
                    3.Add adversarial perturbation to embeddings
                    4.Calculate APR loss
            """
            # Backward to get grads
            # this would be the part we change in defining delta, delta = HPN (phi)
            loss.backward(retain_graph=True)
            grad_u = u.grad
            grad_i = i.grad
            grad_j = j.grad

            # Construct adversarial perturbation based on gradient of loss function, and normalize it with epsilon * norm
            if grad_u is not None:
                delta_u = nn.functional.normalize(grad_u, p=2, dim=1, eps=self.eps)
            else:
                delta_u = torch.rand(u.size()) ## why we have to do this if grad is none?
            if grad_i is not None:
                delta_i = nn.functional.normalize(grad_i, p=2, dim=1, eps=self.eps)
            else:
                delta_i = torch.rand(i.size())
            if grad_j is not None:
                delta_j = nn.functional.normalize(grad_j, p=2, dim=1, eps=self.eps)
            else:
                delta_j = torch.rand(j.size())

            # Add adversarial perturbation to embeddings, now we have q+delta, p+delta
            x_ui_adv = torch.mul(u + delta_u, i + delta_i).sum(dim=1)
            x_uj_adv = torch.mul(u + delta_u, j + delta_j).sum(dim=1)

            # find difference between pos and neg item, then clip value
            x_uij_adv = torch.clamp(x_ui_adv - x_uj_adv,min=-80.0,max=1e8)

            # Calculate APR loss with logsigmoid
            log_prob = F.logsigmoid(x_uij_adv).sum()
            adv_loss = self.reg_adv *(-log_prob) + loss # this is adversarial loss (equation 4 in paper)
            adv_loss.backward()

            return adv_loss

## similar set of with batch size = 512
def evaluate_k(user_emb, item_emb, train_user_list, test_user_list, klist, batch=512):
    """Compute HR and NDCG at k.

    Args:
        user_emb (torch.Tensor): embedding for user [user_num, dim]
        item_emb (torch.Tensor): embedding for item [item_num, dim]
        train_user_list (list(set)):
        test_user_list (list(set)):
        k (list(int)):
    Returns:
        (torch.Tensor, torch.Tensor) HR and NDCG at k
    """

    # Calculate max k value
    max_k = max(klist)
    result = None

    # no iteration = user_num / batch size (which is 512)
    for i in range(0, user_emb.shape[0], batch):

        # Construct mask for each batch

        #new_ones returns a Tensor of size size filled with 1

        # size of the mask vector = (min of batch or user embed) * item+embed
        mask = user_emb.new_ones([min([batch, user_emb.shape[0]-i]), item_emb.shape[0]])
        for j in range(batch):
            if i+j >= user_emb.shape[0]:
                break
            mask[j].scatter_(dim=0, index=torch.tensor(list(train_user_list[i + j])), value=torch.tensor(0.0))

        # Get current result
        cur_result = torch.mm(user_emb[i:i+min(batch, user_emb.shape[0]-i), :], item_emb.t())
        cur_result = torch.sigmoid(cur_result)
        assert not torch.any(torch.isnan(cur_result))

        # Make zero for already observed item
        cur_result = torch.mul(mask, cur_result)
        _, cur_result = torch.topk(cur_result, k=max_k, dim=1)
        result = cur_result if result is None else torch.cat((result, cur_result), dim=0)


    ## basically this chunk collects the results
    result = result.cpu()

    # Sort indice and get HR_NDCG_topk
    HRs, NDCGs = [], []
    for k in klist:
        ndcg, hr = 0, 0
        for i in range(user_emb.shape[0]):
            test = set(test_user_list[i])
            pred = set(result[i, :k].numpy().tolist())
            val = len(test & pred)
            hr += val / max([len(test), 1])
            pred = list(pred)
            x = int(test_user_list[i][0])
            if pred.count(x) != 0:
                position = pred.index(x)
                ndcg += math.log(2) / math.log(position + 2) if position < k else 0
            else:
                ndcg += 0
        NDCGs.append(ndcg / user_emb.shape[0])
        HRs.append(hr / user_emb.shape[0])
        NDCGs.append(ndcg / user_emb.shape[0])
    return HRs, NDCGs


def main(args):
    # Initialize seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

    # Load preprocess data
    with open(args.data, 'rb') as f:
        dataset = pickle.load(f)
        user_size, item_size = dataset['user_size'], dataset['item_size']
        train_user_list, test_user_list = dataset['train_user_list'], dataset['test_user_list']
        train_pair = dataset['train_pair']

    # Create dataset, model, optimizer
    dataset = GetTriplePair(item_size, train_user_list, train_pair, True, args.epochs)

    # load batch of 512 item triplets
    loader = DataLoader(dataset, batch_size=args.batch_size)
    model = MF(user_size, item_size, args.dim, args.reg, args.reg_adv, args.eps)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # Training
    start_time = time.time()
    eval_best_loss = float('inf')
    optimizer.zero_grad()
    epoch = 0
    HR_history = []
    NDCG_history = []
    for u, i, j in loader:
        if epoch in range(args.epochs + args.adv_epoch):
            loss = model(u, i, j, epoch)
            optimizer.step()
            HR_list, NDCG_list = evaluate_k(model.W.detach(),
                                                               model.H.detach(),
                                                               train_user_list,
                                                               test_user_list,
                                                               klist=[50, 100])
            if epoch % args.verbose == (args.verbose - 1):
                if epoch in range(args.epochs):
                    print('BPR-MF Epoch [{}/{}]'.format(epoch + 1, args.epochs + args.adv_epoch))
                if epoch in range(args.epochs, args.adv_epoch + args.epochs):
                    print('AMF Epoch [{}/{}]'.format(epoch + 1, args.epochs + args.adv_epoch))
                print('loss: %.4f' % loss)
                print('HR@50: %.4f, HR@100: %.4f, NDCG@50: %.4f, NDCG@100: %.4f' % (
                    HR_list[0], HR_list[1], NDCG_list[0], NDCG_list[1]))
            HR_history.append(HR_list[1])
            NDCG_history.append(NDCG_list[1])
            if epoch % 100 == 0:
                if loss < eval_best_loss:
                    eval_best_loss = loss
                    dirname = os.path.dirname(os.path.abspath(args.model))
                    os.makedirs(dirname, exist_ok=True)
                    torch.save(model.state_dict(), args.model)
                    time_dif = get_time_dif(start_time)
                    print("time", time_dif)
            epoch += 1
        else:
            break

    fig_HR = plt.figure(edgecolor='blue')
    ax1 = fig_HR.add_subplot(111)
    plt.ylabel('HR@100')
    plt.xlabel('Epoch')
    plt.title('Yelp')
    ax1.plot(range(len(HR_history)), HR_history, c=np.array([255, 71, 90]) / 255.)
    plt.show()
    fig_P = plt.figure(edgecolor='blue')
    ax1 = fig_P.add_subplot(111)
    plt.ylabel('NDCG@100')
    plt.xlabel('Epoch')
    plt.title('Yelp')
    ax1.plot(range(len(NDCG_history)), NDCG_history, c=np.array([255, 71, 90]) / 255.)
    plt.show()


if __name__ == '__main__':
    # Parse argument
    parser = argparse.ArgumentParser()
    parser.add_argument('--data',
                        type=str,
                        default=os.path.join('preprocessed', 'ml-1m.pickle'),
                        help="File path for data")
    # Seed
    parser.add_argument('--seed',
                        type=int,
                        default=0,
                        help="Seed (For reproducability)")
    # Model, default embedding size = 64
    parser.add_argument('--dim',
                        type=int,
                        default=64,
                        help="Dimension for embedding")
    # Optimizer, learning rate is different @ 0.00025
    parser.add_argument('--lr',
                        type=float,
                        default= 0.00025,
                        help="Learning rate")
    parser.add_argument('--reg',
                        type=float,
                        default=0,
                        help="Regularization for user and item embeddings.")
    # Training
    parser.add_argument('--epochs',
                        type=int,
                        default=1000,
                        help="Number of epoch during training")
    parser.add_argument('--batch_size',
                        type=int,
                        default=2000,
                        help="Batch size in one iteration")
    parser.add_argument('--verbose',
                        type=int,
                        default=20,
                        help="Evaluate per X epochs")
    parser.add_argument('--eval_every',
                        type=int,
                        default=20,
                        help="Period for evaluating precision and recall during training")
    parser.add_argument('--model',
                        type=str,
                        default=os.path.join('output', 'bpr.pt'),
                        help="File path for model")
    #original paper has default regularization coefficient and epsilon = 1 and 0.5 respectively

    parser.add_argument('--reg_adv', type=float, default=1,
                        help='Regularization for adversarial loss')
    parser.add_argument('--adv_epoch', type=int, default=1000,
                        help='Add APR in epoch X, when adv_epoch is 0, it\'s equivalent to pure AMF.\n '
                             'And when adv_epoch is larger than epochs, it\'s equivalent to pure MF model. ')
    parser.add_argument('--eps', type=float, default=0.5,
                        help='Epsilon for adversarial weights.')
    args = parser.parse_args()
    main(args)


usage: ipykernel_launcher.py [-h] [--data DATA] [--seed SEED] [--dim DIM]
                             [--lr LR] [--reg REG] [--epochs EPOCHS]
                             [--batch_size BATCH_SIZE] [--verbose VERBOSE]
                             [--eval_every EVAL_EVERY] [--model MODEL]
                             [--reg_adv REG_ADV] [--adv_epoch ADV_EPOCH]
                             [--eps EPS]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/vuhoang/Library/Jupyter/runtime/kernel-86889cae-b42c-46b5-a26c-f7149c78243c.json


SystemExit: 2

/Users/vuhoang/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
model.parameters()

NameError: name 'model' is not defined